In [1]:
import torch.nn.functional as F
import torch
import pandas as pd
import os

In [2]:
num_classes = 9
input_chunk_length = 120

In [3]:
def dscovr_df(filename):
    df = pd.read_csv(os.path.join(os.path.curdir, "data", filename))
    df = df.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
    df.index = df["0"]
    return df.drop(columns="0")
# print(dscovr_df("data_2016.csv")["k_index_target"].unique().size)

In [4]:
from dscovr_dataset import create_dscovr_dataset
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import pandas as pd

train_df = dscovr_df("data_2017.csv")
X_train, y_train = create_dscovr_dataset(train_df.to_numpy(), lookback=input_chunk_length, num_classes=num_classes)
data_train = data.TensorDataset(X_train, y_train)

test_df = dscovr_df("data_2021.csv")
X_test, y_test = create_dscovr_dataset(test_df.to_numpy(), lookback=input_chunk_length, num_classes=num_classes)
data_test = data.TensorDataset(X_test, y_test)

batch_size = 32
train_loader = data.DataLoader(data_train, shuffle=True, batch_size=batch_size)
test_loader = data.DataLoader(data_test, shuffle=True, batch_size=batch_size * 8)

100%|██████████| 10000/10000 [00:00<00:00, 36810.51it/s]


In [5]:
from transformer_classifier import TransformerClassifier
model = TransformerClassifier(
    num_classes=num_classes,
    input_chunk_length = input_chunk_length,
    d_model = 64,
    input_dim=54,
    nhead = 8,
    dim_feedforward = 1024,
    num_layers = 3,
    dropout = 0.5,
    activation = "relu",
    classifier_dropout = 0.4,
).to(device="cuda")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
from tqdm import tqdm
from torchmetrics.functional import precision, recall

n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    prec, rec = [], []
    run_loss = 0
    for X_batch, y_batch in tqdm(train_loader):
        X_batch, y_batch = X_batch.to(device="cuda"), y_batch.to(device="cuda", dtype=torch.float)
        y_pred = model(X_batch)

        optimizer.zero_grad()
        loss = loss_fn(y_pred, y_batch)

        y_pred = y_pred.argmax(dim=1)
        y_batch = y_batch.argmax(dim=1)
        curr_prec = precision(y_pred, y_batch, "multiclass", num_classes=num_classes, average="micro")
        curr_rec = recall(y_pred, y_batch, "multiclass", num_classes=num_classes, average="micro")
        run_loss += loss.item()
        prec.append(curr_prec.item())
        rec.append(curr_rec.item())

        loss.backward()
        optimizer.step()
    print(f"For index {epoch}: precision: {sum(prec)/len(prec)} recall: {sum(rec)/len(rec)} loss: {run_loss / len(train_loader)}")
    print("--------------------------------")

    model.eval()
    if epoch % 5 == 4:
        prec, rec = [], []
        run_loss = 0
        with torch.no_grad():
            for i, (inputs, targets) in tqdm(enumerate(test_loader)):
                inputs, targets = inputs.to(device="cuda"), targets.to(device="cuda")
                y_pred = model(inputs)

                y_pred = y_pred.argmax(dim=1)
                targets = targets.argmax(dim=1)
                curr_prec = precision(y_pred, targets, "multiclass", num_classes=num_classes, average="micro")
                curr_rec = recall(y_pred, targets, "multiclass", num_classes=num_classes, average="micro")
                run_loss += loss.item()
                prec.append(curr_prec.item()), rec.append(curr_rec.item())

        # Print accuracy
        print(f"For test: precision: {sum(prec)/len(prec)} recall: {sum(rec)/len(rec)} loss: {run_loss / len(test_loader)}")
        print("--------------------------------")

100%|██████████| 313/313 [00:29<00:00, 10.61it/s]


For index 0: precision: 0.240814696485623 recall: 0.240814696485623 loss: 2.074426086900423
--------------------------------


100%|██████████| 313/313 [00:29<00:00, 10.61it/s]


For index 1: precision: 0.24930111821086262 recall: 0.24930111821086262 loss: 2.069230978243413
--------------------------------


100%|██████████| 313/313 [00:29<00:00, 10.60it/s]


For index 2: precision: 0.24930111821086262 recall: 0.24930111821086262 loss: 2.0674910128535386
--------------------------------


100%|██████████| 313/313 [00:29<00:00, 10.56it/s]


For index 3: precision: 0.2437100638977636 recall: 0.2437100638977636 loss: 2.0731636860432148
--------------------------------


 60%|██████    | 188/313 [00:17<00:11, 10.52it/s]

In [ ]:
from torchmetrics.functional import precision
preds  = torch.tensor([[1, 0], [1, 0], [1, 0], [1, 0]])
target = torch.tensor([[1, 0], [1, 0], [1, 0], [1, 0]])
print(precision(preds, target, task="multiclass", average='micro', num_classes=2))
# precision(preds, target, average='micro')